# 간단한 컨브넷 만들기

In [1]:
#keras를 아나콘다에서 다운 받아야댐
from keras import layers
from keras import models
import numpy as np
import pandas as pd

model=models.Sequential() #케라스 레이어 객체 생성
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1))) # 26 * 26 * 32
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [2]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [3]:
model.add(layers.Flatten()) # fully connected layer
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

# Mnist 이미지 컨브넷에 학습

In [5]:
# keras 함수들과 관련되서는 사이트 https://keras.io/utils/ 에 들어가면 나와있다.
from keras.datasets import mnist  #keras.datasets에는 Cifar10의 부분 이미지들과 IMDB,
#MNIST, Fashion-MNIST, Boston housing 데이터가 들어있다.
from keras.utils import to_categorical # class vector를 이진 행렬로 바꾸어줌,
#categorical_crossentrophy 사용을 위해 보통 써줌

(train_images, train_labels), (test_images, test_labels)= mnist.load_data()

train_images=train_images.reshape((60000, 28, 28, 1)) # 60000만개 데이터를 28* 28* 1 로 받음 (흑백이라)
train_images=train_images.astype('float32') / 255  #픽셀은 0~255값이므로 이를 전부 0~1값으로 만들어주기위해 

test_images=test_images.reshape((10000, 28, 28, 1))
test_images=test_images.astype('float32') / 255

train_labels=to_categorical(train_labels)
test_labels=to_categorical(test_labels)

model.compile(optimizer='rmsprop',  #학습시킬 모델에 대해 정해주는것
             loss='categorical_crossentropy',   #
             metrics=['accuracy'])              # 메트릭스는 결과가 몇개인지를 나타낼때
 #결과가 2개일때 ex)metrics={'output_a': 'accuracy', 'output_b': ['accuracy', 'mse']} 

model.fit(train_images, train_labels, epochs=5, batch_size=64)
# epochs 는 전체의 학습과정 수, batch_size는 한번 그레디언트 업데이트할때까지 입력샘플수

test_loss, test_acc=model.evaluate(test_images, test_labels) # ()안의 자료들의 로스값과 매트릭스값 반환
test_loss
test_acc

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
60000/60000 [==============================] - 19s 320us/step - loss: 0.1807 - acc: 0.9435
Epoch 2/5
60000/60000 [==============================] - 20s 329us/step - loss: 0.0469 - acc: 0.9853
Epoch 3/5
60000/60000 [==============================] - 20s 330us/step - loss: 0.0325 - acc: 0.9895
Epoch 4/5
60000/60000 [==============================] - 20s 329us/step - loss: 0.0235 - acc: 0.9924
Epoch 5/5
10000/10000 [==============================] - 1s 122us/step


0.9892

In [7]:
test_loss

0.033060991139275936

In [8]:
model_no_max_pool=models.Sequential()
model_no_max_pool.add(layers.Conv2D(32, (3, 3), activtivation='relu', input_shape=(28, 28, 1)))
model_no_max_pool.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_no_max_pool.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [9]:
model_no_max_pool.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 22, 22, 64)        36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


### 최대 풀링 쓰는 이유 위 model_no_max_pool 모델을 보면서 이해 
1. 특성의 공간적 계층 구조 학습을 못하고 초기 입력의 7 by 7 윈도우 영역에 대한 정보만 담고있다.-> (28, 28, 3)이 3 by 3 윈도우 두번을 거치면서 (22, 22, 64)가 되었는데 이는 처음에 7 by 7 윈도우를
한번 거친것과 같다. 즉 패턴정보가 많이 부족


2. 다운 샘플링을 통해 가충치수를 줄이고 특징을 추출하여 대체함 이를 안할시 가중치가 너무 많아서 계산량도 많고 심각한 오버피팅이 일어날것임


#### 그렇다면 왜 풀링의 종류가 많은대 최대 풀링을 쓸까?
최대풀링이 어떤 패턴이나 개념의 존재여부를 잘 나타내고 잘 작동하기 때문